In [88]:
#! /usr/bin/env python3
# coding: utf-8
from reader import read
from alpha import Alpha
from petri_net import PetriNet
from subprocess import check_call
import sys
import os.path
import itertools
import copy

log = read('/Projects/misha/pyalpha-master/tests/test2.txt')
filename = 'filename'
alpha_model = Alpha(log)
alpha_model.generate_footprint(txtfile="{}_footprint.txt".format(filename))
pn = PetriNet()
pn.generate_with_alpha(alpha_model, dotfile="{}.dot".format(filename))
check_call(["dot", "-Tpng", "{}.dot".format(filename),"-o", "{}.png".format(filename)])

0

In [60]:
for i in itertools.permutations(alpha_model.tl,2):
    print(i)

('b', 'a')
('b', 'd')
('b', 'c')
('b', 'e')
('a', 'b')
('a', 'd')
('a', 'c')
('a', 'e')
('d', 'b')
('d', 'a')
('d', 'c')
('d', 'e')
('c', 'b')
('c', 'a')
('c', 'd')
('c', 'e')
('e', 'b')
('e', 'a')
('e', 'd')
('e', 'c')


In [76]:
alpha_model.ds

{('a', 'b'),
 ('a', 'c'),
 ('a', 'e'),
 ('b', 'c'),
 ('b', 'd'),
 ('c', 'b'),
 ('c', 'd'),
 ('e', 'd')}

In [85]:
def direct_succession():
        # x > y
        ds = set()
        for trace in alpha_model.log:
            for x, y in zip(trace, trace[1:]):
                ds.add((x, y))
        return ds

def causality_parallel_choice(ds):
        # x -> y
        cs = set()
        pr = set()
        ind = set()
        all_pair = itertools.permutations(alpha_model.tl,2)
        
        for pair in ds:
            if pair[::-1] not in ds:
                cs.add(pair)
            else:
                pr.add(pair)
                
        for pair in all_pair:
            if pair not in cs and pair[::-1] not in cs and pair not in pr:
                ind.add(pair)

        return cs,pr,ind

def get_XL_set(tl, ind, cs):
        xl = set()
        subsets = itertools.chain.from_iterable(itertools.combinations(tl, r) for r in range(1, len(tl) + 1))
        independent_a_or_b = [a_or_b for a_or_b in subsets if is_ind_set(a_or_b, ind)]
        for a, b in itertools.product(independent_a_or_b, independent_a_or_b):
            if is_cs_set((a, b), cs):
                xl.add((a, b))
        return xl

cs,pr,ind = causality_parallel_choice(alpha_model.direct_succession())
xl= get_XL_set(alpha_model.tl,ind,cs)
xl

{(('a',), ('b',)),
 (('a',), ('b', 'e')),
 (('a',), ('c',)),
 (('a',), ('c', 'e')),
 (('a',), ('e',)),
 (('b',), ('d',)),
 (('b', 'e'), ('d',)),
 (('c',), ('d',)),
 (('c', 'e'), ('d',)),
 (('e',), ('d',))}

In [81]:
s

In [86]:
for i in itertools.chain.from_iterable(itertools.combinations(alpha_model.tl, r) for r in range(1, len(alpha_model.tl) + 1)):
    print(i)
    
independent_a_or_b = [a_or_b for a_or_b in itertools.chain.from_iterable(itertools.combinations(alpha_model.tl, r) for r in range(1, len(alpha_model.tl) + 1)) if is_ind_set(a_or_b, ind)]
print("SSS")
for i in itertools.product(independent_a_or_b, independent_a_or_b):
    print(i)

('b',)
('a',)
('d',)
('c',)
('e',)
('b', 'a')
('b', 'd')
('b', 'c')
('b', 'e')
('a', 'd')
('a', 'c')
('a', 'e')
('d', 'c')
('d', 'e')
('c', 'e')
('b', 'a', 'd')
('b', 'a', 'c')
('b', 'a', 'e')
('b', 'd', 'c')
('b', 'd', 'e')
('b', 'c', 'e')
('a', 'd', 'c')
('a', 'd', 'e')
('a', 'c', 'e')
('d', 'c', 'e')
('b', 'a', 'd', 'c')
('b', 'a', 'd', 'e')
('b', 'a', 'c', 'e')
('b', 'd', 'c', 'e')
('a', 'd', 'c', 'e')
('b', 'a', 'd', 'c', 'e')
SSS
(('b',), ('b',))
(('b',), ('a',))
(('b',), ('d',))
(('b',), ('c',))
(('b',), ('e',))
(('b',), ('b', 'e'))
(('b',), ('a', 'd'))
(('b',), ('c', 'e'))
(('a',), ('b',))
(('a',), ('a',))
(('a',), ('d',))
(('a',), ('c',))
(('a',), ('e',))
(('a',), ('b', 'e'))
(('a',), ('a', 'd'))
(('a',), ('c', 'e'))
(('d',), ('b',))
(('d',), ('a',))
(('d',), ('d',))
(('d',), ('c',))
(('d',), ('e',))
(('d',), ('b', 'e'))
(('d',), ('a', 'd'))
(('d',), ('c', 'e'))
(('c',), ('b',))
(('c',), ('a',))
(('c',), ('d',))
(('c',), ('c',))
(('c',), ('e',))
(('c',), ('b', 'e'))
(('c',), (

In [52]:
def is_ind_set(s,ind):
    if len(s) == 1:
        return True
    else:
        s_all = itertools.combinations(s,2)
        for pair in s_all:
            if pair not in alpha_model.ind:
                return False
        return True
    
def is_cs_set(s, cs):
    set_a, set_b = s[0], s[1]
    s_all = itertools.product(set_a, set_b)
    for pair in s_all:
        if pair not in cs:
            return False
    return True